```python
# input_data shape
Input: (batch_size, in_channel, width, height)
# conv layer
class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
input: (Batch_size, C_in, H_in, W_in)
output: (Batch_size, C_out, H_out, W_out)

weight(tensor): (out_channels, in_channels,kernel_size)
bias(tensor): (out_channel)
```

In [ ]:
from itertools import product

import time
import numpy as np
from sklearn import datasets
from scipy.stats import multivariate_normal as mvn
import matplotlib.pyplot as plt

from ripser import Rips
from persim import PersistenceImager

import os
import math

import seaborn
import pandas as pd
import torch
import random
from torch import nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import re
import cv2
from PIL import Image

KeyboardInterrupt: 

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
# parameters
learning_rate = 0.001
training_epochs = 600
batchsize = 25
testbatchsize = 15


In [ ]:
# dir = os.listdir("./metric")
# data = []
# a = []
# for metric in dir:
#     a = re.findall("\d+\.?\d*", metric)
#     data.append([np.loadtxt("./metric/"+metric), metric])
#     # print(a[-2])
#     print(metric[-2])


In [ ]:
class My_dataset(Dataset):
    def __init__(self, train):
        super().__init__()
        # 使用sin函数返回10000个时间序列,如果不自己构造数据，就使用numpy,pandas等读取自己的数据为x即可。
        # 以下数据组织这块既可以放在init方法里，也可以放在getitem方法里
        dir = os.listdir("./grids_trainset")
        dirtest = os.listdir("./grids_testset")
        data = []
        a = []
        if train:
            # for metric in dir:
            #     data.append(
            #         [np.loadtxt("./grids_trainset_cifar/"+metric), metric])
            sample = []
            for metric in dir:
                sample.append([np.loadtxt("./grids_trainset/"+metric), metric])
                if len(sample) == 4:
                    res = cv2.merge([i[0] for i in sample])
                    res = np.transpose(res,(2,0,1))
                    data.append([res, metric])
                    sample = []
            
        else:
            # for metric in dirtest:
            #     data.append(
            #         [np.loadtxt("./grids_testset_cifar/"+metric), metric])
            sample = []
            for metric in dirtest:
                sample.append([np.loadtxt("./grids_testset/"+metric), metric])
                if len(sample) == 4:
                    res = cv2.merge([i[0] for i in sample])
                    res = np.transpose(res,(2,0,1))
                    data.append([res, metric])
                    sample = []

        # data = [item for item in data if (int(re.findall("\d+\.?\d*", item[1])[0]) == 0 or int(re.findall("\d+\.?\d*", item[1])[0]) > 1)]
        self.x = [item[0] for item in data]
        self.y = [math.ceil(int(re.findall("\d+\.?\d*", item[1])[0]) /
                            (int(re.findall("\d+\.?\d*", item[1])[0])+1)) for item in data]
        # self.y = [int(re.findall("\d+\.?\d*", item[1])[0]) for item in data]
        self.src,  self.trg = [], []
        for i in range(len(data)):
            self.src.append(self.x[i])
            self.trg.append(self.y[i])

    def __getitem__(self, index):
        return self.src[index], self.trg[index]

    def __len__(self):
        return len(self.src)

 # 或者return len(self.trg), src和trg长度一样


In [ ]:
data_tf = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])
     
data_train = My_dataset(train=True)
data_test = My_dataset(train=False)
data_loader_train = DataLoader(data_train, batch_size=batchsize, shuffle=True)
data_loader_test = DataLoader(data_test, batch_size=testbatchsize, shuffle=True)


# i_batch的多少根据batch size和def __len__(self)返回的长度确定
# batch_data返回的值根据def __getitem__(self, index)来确定
# 对训练集：(不太清楚enumerate返回什么的时候就多print试试)
print("train")
for i_batch, batch_data in enumerate(data_loader_train):
    print(i_batch)  # 打印batch编号
    print(batch_data[0].shape)  # 打印该batch里面src
    print(batch_data[1])  # 打印该batch里面trg
# # 对测试集：（下面的语句也可以）
print("test")
for i_batch, (src, trg) in enumerate(data_loader_test):
    print(i_batch)  # 打印batch编号
    print(src)  # 打印该batch里面src的尺寸
    print(trg)  # 打印该batch里面trg的尺寸



train
0
torch.Size([25, 4, 128, 128])
tensor([1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0,
        1])
1
torch.Size([25, 4, 128, 128])
tensor([1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        0])
2
torch.Size([25, 4, 128, 128])
tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
        1])
3
torch.Size([25, 4, 128, 128])
tensor([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
        1])
4
torch.Size([25, 4, 128, 128])
tensor([0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
        1])
test
0
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0.,

In [ ]:
# class MyNet(torch.nn.Module):
#     def __init__(self):
#         super(MyNet, self).__init__()

#         self.fc = torch.nn.Sequential(
#             torch.nn.Linear(256*256, 1024),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(1024, 1024),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(1024, 128),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(128, 64),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(64, 2)
#         )

#     def forward(self, x):
#         x = x.view(-1,256*256)
#         x = self.fc(x)
#         return x


# model = MyNet().to(device)

In [ ]:
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(4, 16, kernel_size=64),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(inplace=True)
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(16 * 32 * 32, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


model = MyNet().to(device)

In [ ]:
# define cost/loss & optimizer
# Softmax is internally computed.
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [ ]:
total_batch = len(data_loader_train)

for epoch in range(training_epochs):
    model.train()
    avg_cost = 0

    # for X, Y in data_loader_train:
    #     X = torch.autograd.Variable(X).to(device).float()
    #     Y = Y.to(device)

    #     optimizer.zero_grad()
    #     hypothesis = model(X)
    #     cost = criterion(hypothesis, Y)
    #     cost.backward()
    #     optimizer.step()

    #     avg_cost += cost / total_batch
    # model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(data_loader_train):
        inputs, targets = inputs.to(device).float(), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        avg_cost += loss / total_batch

    
    # if avg_cost < 0.1:
    #     break
    model.eval()
    # with torch.no_grad():
    #     for X_test, Y_test in data_loader_test:
    #         X_test = X_test.to(device).float()
    #         Y_test = Y_test.to(device)

    #     prediction = model(X_test)
    #     correct_prediction = torch.argmax(prediction, 1) == Y_test
    #     accuracy = correct_prediction.float().mean()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader_train):
            inputs, targets = inputs.to(device).float(), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    trainacc = 100.*correct/total
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader_test):
            inputs, targets = inputs.to(device).float(), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    testacc = 100.*correct/total
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 'trainacc:', trainacc, 'testacc:', testacc)

print('Learning finished')


Epoch: 0001 cost = 0.692817569 trainacc: 50.4 testacc: 50.0
Epoch: 0002 cost = 0.692546427 trainacc: 50.4 testacc: 50.0
Epoch: 0003 cost = 0.692036808 trainacc: 50.4 testacc: 50.0
Epoch: 0004 cost = 0.691783845 trainacc: 50.4 testacc: 50.0
Epoch: 0005 cost = 0.691407800 trainacc: 50.4 testacc: 50.0
Epoch: 0006 cost = 0.691054165 trainacc: 50.4 testacc: 50.0
Epoch: 0007 cost = 0.690825343 trainacc: 50.4 testacc: 50.0
Epoch: 0008 cost = 0.690489948 trainacc: 50.4 testacc: 50.0
Epoch: 0009 cost = 0.690196335 trainacc: 50.4 testacc: 50.0
Epoch: 0010 cost = 0.689880252 trainacc: 50.4 testacc: 50.0
Epoch: 0011 cost = 0.689448655 trainacc: 50.4 testacc: 50.0
Epoch: 0012 cost = 0.689189315 trainacc: 50.4 testacc: 50.0
Epoch: 0013 cost = 0.688831568 trainacc: 50.4 testacc: 50.0
Epoch: 0014 cost = 0.688656807 trainacc: 50.4 testacc: 50.0
Epoch: 0015 cost = 0.688206792 trainacc: 50.4 testacc: 50.0
Epoch: 0016 cost = 0.688012302 trainacc: 50.4 testacc: 50.0
Epoch: 0017 cost = 0.687527239 trainacc:

In [ ]:
# Test the model using test sets
# model.load_state_dict(torch.load(".\merge_save\merge_attack0normal10_05-07--21-10-32.pth")["state_dict"])
# model.load_state_dict(torch.load(r".\myfed_normal_save\model18-24-02.pth")["state_dict"])
model.eval()
with torch.no_grad():
    for X_test, Y_test in data_loader_test:
        X_test = X_test.to(device).float()
        Y_test = Y_test.to(device)

        prediction = model(X_test)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        print(prediction,correct_prediction, Y_test)
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

    # Get one and predict
    # r = random.randint(0, len(mnist_test) - 1)
    # X_single_data = mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    # Y_single_data = mnist_test.targets[r:r + 1].to(device)

    # print('Label: ', Y_single_data.item())
    # single_prediction = model(X_single_data)
    # print('Prediction: ', torch.argmax(single_prediction, 1).item())


tensor([[-3.3377e+00,  2.9925e+00],
        [-2.8413e+00,  2.5115e+00],
        [-3.2303e+00,  2.7332e+00],
        [-5.4596e-04, -2.9934e-01],
        [-6.4422e-01,  4.5668e-01],
        [-6.4423e-01,  3.0870e-01],
        [-7.5181e-01,  4.2383e-01],
        [-3.3621e+00,  2.9561e+00],
        [-3.2106e+00,  2.8649e+00],
        [-1.7504e+00,  1.3910e+00],
        [ 9.1613e-01, -9.9515e-01],
        [ 2.7600e+00, -2.8394e+00],
        [-8.8490e-01,  6.3188e-01],
        [-4.9878e-01, -9.9034e-03],
        [-2.6865e+00,  2.3277e+00]], device='cuda:0') tensor([ True,  True, False, False,  True,  True, False,  True, False,  True,
         True,  True, False,  True,  True], device='cuda:0') tensor([1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1], device='cuda:0')
Accuracy: 0.6666666865348816
tensor([[-0.9546,  0.5854],
        [ 2.0866, -2.0870],
        [ 0.6185, -0.8793],
        [ 3.0498, -3.0567],
        [ 2.2323, -2.3653],
        [-3.0995,  2.7563],
        [-1.9152,  1.5265],
       